In [1]:
import pandas as pd
df = pd.read_csv("../data/listings_al.csv", low_memory=False)

In [3]:
columns = ['location_id', 'street_full', 'street_name', 'street_type', 'parish', 'county', 'zip_code', 'coastal_area']
df_loc = {}

In [5]:
df_loc['location_id'] = [value for value in df_property['id']]
df_loc['street_full'] = [get_property_type_category(value) for value in df_property['property_type']]
df_loc['street_name'] = [value for value in df_property['property_type']]
df_loc['street_type'] = [value for value in df_property['room_type']]
df_loc['parish'] = [get_accommodates(value) for value in df_property['accommodates']]
df_loc['county'] = [get_bathrooms(value) for value in df_property['bathrooms']]
df_loc['zip_code'] = [get_bedrooms(value) for value in df_property['bedrooms']]
df_loc['coastal_area'] = [value for value in df_property['bed_type']]

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [6]:
from streetaddress import StreetAddressParser

addr_parser = StreetAddressParser()



addr = addr_parser.parse(df['Endereco'][0])
print (addr['street_full'])
print (addr['street_name'])
print (addr['street_type'])

Calçada Calçada São Vicente 4 2
Calçada Calçada São Vicente 4 2
None


In [5]:
df['Endereco'][0]

'Calçada Calçada São Vicente 4 2 '